Accuracy of fold 1: 0.9733333333333334
Accuracy of fold 2: 0.9466666666666667
Average accuracy: 0.96


Cross-validation scores: [0.96 0.92]
Average score: 0.94
Std deviation of scores: 0.019999999999999962


Scikit-Learn implements several cross-validation schemes that are useful in particular situations; these are implemented through iterators in the `sklearn.model_selection` module. In practice, we have to define the iterator through the `cv` parameter.

As an example, we might want to go to the extreme case where the number of subsets is equal to the number of examples contained in the training set. In this case, in each iteration, a model is fitted using all but one example, and accuracy is measured against the only example not used during training. This type of cross-validation is known as **leave-one-out cross-validation**.

The example below illustrates the use of this particular case leave-one-out cross-validation. Since there are 150 samples, the leave-one-out cross-validation produces 150 estimates (1.0 indicates a successful forecast, and 0.0 indicates an unsuccessful one).

In [3]:
from sklearn.model_selection import LeaveOneOut

scores = cross_val_score(model, X, y, cv=LeaveOneOut())
scores

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 0., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

The average of these 150 values ​​corresponds to the estimated error rate of the model produced:

In [4]:
print('Average score: ', scores.mean())
print('Std deviation of scores: ', scores.std())

Average score:  0.96
Std deviation of scores:  0.19595917942265423


## Stratified k-fold cross-validation

In the context of the classification task, an unbalanced dataset presents unequal proportions of examples for each class. See the image below ([source](https://www.researchgate.net/publication/306376881_Survey_of_resampling_techniques_for_improving_classification_performance_in_unbalanced_datasets)) for an example in the binary classification case.

<p align="center">
<img src="https://miro.medium.com/max/698/1*cd6AorHoJYMFyj7IZd2nOg.png">
</p>

For unbalanced datasets, if the vanilla version of $k$-fold cross-validation is used, it can be the case that one or more produced folds end up with zero examples of the **minority class**. In such situations, it is adequate to use *stratified $k$-fold cross-validation*. In stratified $k$ cross-validation, class proportions are preserved in each of the $k$ subsets, so that each one is representative of the class proportions in the training data set. The image below (source) illustrates how stratified $k$-fold crors-validation works.

<p align="center">
<img src="https://miro.medium.com/max/562/0*QKJTHrcriSx2ZNYr.png">
</p>

In Scikit-Learn, the [`StratifiedKFold`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.StratifiedKFold.html) class implements the stratified cross-validation method. See the following example, that applies stratified cross-validation to a toy dataset with two classes. Notice that the proportions os classes in each fold is approximately the same as the original dataset.

In [5]:
import numpy as np
from sklearn.model_selection import StratifiedKFold

# data matrix
X = np.array([[1, 2], [3, 4], [1, 2], [3, 4], [5, 6], [7, 8],
              [9, 10], [9, 10], [11, 12], [11, 12],
              [1, 2], [3, 4], [5, 6], [7, 8], [4, 5], [5, 6], [7, 8], [4, 5]])

# response vector
y = np.array([0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

skf = StratifiedKFold(n_splits=2)
skf.get_n_splits(X, y)

print(skf)

# print the indices of examples in each fold
for train_index, test_index in skf.split(X, y):
   print("TRAIN:", train_index, "TEST:", test_index)
   X_train, X_test = X[train_index], X[test_index]
   y_train, y_test = y[train_index], y[test_index]

StratifiedKFold(n_splits=2, random_state=None, shuffle=False)
TRAIN: [ 3  4  5 12 13 14 15 16 17] TEST: [ 0  1  2  6  7  8  9 10 11]
TRAIN: [ 0  1  2  6  7  8  9 10 11] TEST: [ 3  4  5 12 13 14 15 16 17]


In [15]:
import numpy as np

from time import time
from scipy.stats import randint as sp_randint

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.datasets import load_digits
from sklearn.ensemble import RandomForestClassifier

# get some data
digits = load_digits()
X, y = digits.data, digits.target

# build a classifier
clf = RandomForestClassifier(n_estimators=20)

# Utility function to report best scores
def report(results, n_top=3):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.3f} (std: {1:.3f})".format(
                  results['mean_test_score'][candidate],
                  results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")


# specify parameters and distributions to sample from
param_dist = {"max_depth": [3, None],
              "max_features": sp_randint(1, 11),
              "min_samples_split": sp_randint(2, 11),
              "bootstrap": [True, False],
              "criterion": ["gini", "entropy"]}

# run randomized search
n_iter_search = 20
random_search = RandomizedSearchCV(clf, param_distributions=param_dist,
                                   n_iter=n_iter_search, cv=5)

start = time()
random_search.fit(X, y)
print("RandomizedSearchCV took %.2f seconds for %d candidates"
      " parameter settings." % ((time() - start), n_iter_search))
report(random_search.cv_results_)

# use a full grid over all parameters
param_grid = {"max_depth": [3, None],
              "max_features": [1, 3, 10],
              "min_samples_split": [2, 3, 10],
              "bootstrap": [True, False],
              "criterion": ["gini", "entropy"]}

# run grid search
grid_search = GridSearchCV(clf, param_grid=param_grid, cv=5)
start = time()
grid_search.fit(X, y)

print("GridSearchCV took %.2f seconds for %d candidate parameter settings."
      % (time() - start, len(grid_search.cv_results_['params'])))
report(grid_search.cv_results_)

RandomizedSearchCV took 7.79 seconds for 20 candidates parameter settings.
Model with rank: 1
Mean validation score: 0.930 (std: 0.032)
Parameters: {'bootstrap': False, 'criterion': 'gini', 'max_depth': None, 'max_features': 9, 'min_samples_split': 9}

Model with rank: 2
Mean validation score: 0.925 (std: 0.027)
Parameters: {'bootstrap': True, 'criterion': 'gini', 'max_depth': None, 'max_features': 3, 'min_samples_split': 3}

Model with rank: 3
Mean validation score: 0.923 (std: 0.031)
Parameters: {'bootstrap': True, 'criterion': 'gini', 'max_depth': None, 'max_features': 10, 'min_samples_split': 6}

GridSearchCV took 31.68 seconds for 72 candidate parameter settings.
Model with rank: 1
Mean validation score: 0.934 (std: 0.020)
Parameters: {'bootstrap': False, 'criterion': 'entropy', 'max_depth': None, 'max_features': 10, 'min_samples_split': 2}

Model with rank: 2
Mean validation score: 0.932 (std: 0.022)
Parameters: {'bootstrap': False, 'criterion': 'gini', 'max_depth': None, 'max_fe

# Model selection: practical tips

* It is appropriate to start with a simple algorithm that can be used quickly.

* Use learning curves to decide for more (or less) data, more (or less) attributes, etc.

* Another technique used in model selection is *error analysis*, which corresponds to manually checking the examples (in the validation set) that the algorithm has erred. It is sometimes possible to detect a pattern of systematic error in these examples.

* The motivation for using k-fold cross-validation is to be able to use a good amount of test or validation data, without significantly reducing the training data set. In situations where there is enough data to have a good-sized training data set, in addition to reasonable amounts of test and validation data, holdout cross validation can be used.